First step is to change the directory which should be done after selecting the working kernal 

In [14]:
import os

In [15]:
%pwd

'c:\\Users\\rishi\\Desktop\\data\\kaggle\\NLPTextSummarization\\NLPTextSummarization'

In [16]:
#os.chdir("../")

In [17]:
%pwd

'c:\\Users\\rishi\\Desktop\\data\\kaggle\\NLPTextSummarization\\NLPTextSummarization'

Defining entity

In [18]:
from dataclasses import dataclass
from pathlib import Path

#this entity will return only 3 things as defined in the config.yaml file
#the root path, status and all required files
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path 
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list


Configuration Manager

In [19]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        #return data_validation_config #similar to the first notebook(data_ingestion)
    
    def get_data_validation_config(self) -> DataValidationConfig: # "DataValidationConfig" is the return type which is created above
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(  #these files will be read from config.yaml file
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config
    

Components

In [21]:
import os
from textSummarizer.logging import logger

In [22]:
#this is similar to DataIngestion created in the first notebook
#it will take the output from the above code block and will be initialised within "self.config"
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config #will be initialized here

    def validate_all_files_exist(self)-> bool: #method to validate file existance
        try:
            validate_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES: #will check if the files are available
                    validate_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validate_status}")
                
                else:
                    validate_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validate_status}")     

            return validate_status #these files will be saved inside data_validation folder

        except Exception as e:
            raise e

In the above code we can also keep column validation criteria, data type etc.

Pipeline

In [23]:
#this code is also similar to the first book's pipeline just its validation
#and will return the validation related output and not download and extract
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2023-12-27 23:32:16,224: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-27 23:32:16,226: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-27 23:32:16,229: INFO: common: created directory at: artifacts]
[2023-12-27 23:32:16,230: INFO: common: created directory at: artifacts/data_validation]
